In [1]:
import numpy as np #Linear algebra and mathematical operations
import pandas as pd #importing and loading data
from sklearn.preprocessing import OneHotEncoder

In [2]:
iris_df = pd.read_csv("/content/drive/MyDrive/Data Science /Iris.csv")
iris_df = iris_df.sample(frac=1).reset_index(drop=True) # Shuffle

In [3]:
iris_df.head()


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,105,6.5,3.0,5.8,2.2,Iris-virginica
1,23,4.6,3.6,1.0,0.2,Iris-setosa
2,107,4.9,2.5,4.5,1.7,Iris-virginica
3,34,5.5,4.2,1.4,0.2,Iris-setosa
4,116,6.4,3.2,5.3,2.3,Iris-virginica


In [4]:

X = iris_df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
X = np.array(X)
X[:5]


array([[6.5, 3. , 5.8, 2.2],
       [4.6, 3.6, 1. , 0.2],
       [4.9, 2.5, 4.5, 1.7],
       [5.5, 4.2, 1.4, 0.2],
       [6.4, 3.2, 5.3, 2.3]])

In [5]:
one_hot_encoder = OneHotEncoder(sparse=False)
Y = iris_df.Species
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1))
Y[:5]

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

In [103]:
class ANN_Classification():

  
    def __init__ (self, hidden_Layer_Size = [100,], learning_Rate = 0.001, epochs = 10):
        self.hidden_Layer_Size = hidden_Layer_Size
        self.learning_Rate = learning_Rate
        self.epochs = epochs
       # self.activation_function = activation_function
        self.weights = None

    
    def sigmoid(self, x, derivative=False):
        if derivative:
            return np.multiply(x, 1-x)
        return 1/(1 + np.exp(-x))
        

    def softmax(self, x):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        return exps / np.sum(exps, axis=0)
            

    #function for forward propogation
    def forward_Prop(self, x, layers):
        activations, layer_input = [x], x
        for j in range(layers):
         # i = 
          #print("i = "+str(i))
          if j <= (layers-1):

            activation = self.sigmoid(np.dot(layer_input, self.weights[j].T))
            activations.append(activation)
            layer_input = np.append(1, activation)

          else :
            activation = self.softmax(np.dot(layer_input, self.weights[j].T))
            activations.append(activation)
            layer_input = np.append(1, activation)

        return activations


    def back_prop(self, y, activations, layers):
      outputFinal = activations[-1]
      error = np.matrix(y - outputFinal) 
      
      # Error after 1 cycle
      for j in range(layers, 0, -1):
        currActivation = activations[j]
       
        if(j > 1):
          # Append previous
          prevActivation = np.append(1, activations[j-1])
        else:
          # First hidden layer
          prevActivation = activations[0]
       
        delta = np.multiply(error, self.sigmoid(currActivation, derivative = True))
        self.weights[j-1] += self.learning_Rate * np.multiply(delta.T, prevActivation)
         
        wc = np.delete(self.weights[j-1], [0], axis=1)
        error = np.dot(delta, wc) #current layer error
       
      return self.weights

    
    def initialize_Weight(self, layers):
      layer, self.weights = len(layers), []
      #for loop to intialize the weight randomly
      for i in range(1, layer):
        #assigning random weights
        w = [[np.random.uniform(-1, 1) for j in range(layers[i-1] + 1)]for k in range(layers[i])]
        self.weights.append(np.matrix(w))
    
      return self.weights
    

    #train function
    def train(self, X, y):
        layers_weights = len(self.weights)
        
        for i in range(len(self.X)):
          x, y = self.X[i], self.y[i]
          x = np.matrix(np.append(1, x))
          
          activations = self.forward_Prop(x, layers_weights)
          self.weights = self.back_prop(y, activations, layers_weights)
          
        return self.weights


    def fit(self, X, y):
        self.X = X
        self.y = y
        hidden_Layers = len(self.hidden_Layer_Size) - 1
        self.weights = self.initialize_Weight(self.hidden_Layer_Size)

        for epoch in range(1, self.epochs+1):
          weights = self.train(self.X, self.y)
          
        return self.weights

        
    def Predict(self, X):
        result = []
        for i in range(len(X)):
          x = X[i]
          #print(str(i) + " " + str(len(X)))
          layers = len(self.weights)
          item = np.append(1, x)

          # Forward prop.
          activations = self.forward_Prop(item, layers)
          
          Final_output = activations[-1].A1
          index = self.FindMaxActivation(Final_output)
          
          predicted = [0 for j in range(len(Final_output))]
          predicted[index] = 1 
      
          
          result.append(predicted)
        
        return result
      
    def FindMaxActivation(self, output):
        m, index = output[0], 0
        for i in range(1, len(output)):
          if(output[i] > m):
            m, index = output[i], i
        
        return index

In [130]:
f = len(X[0]) # no. of features
o = len(Y[0]) # no. of classes

layers = [f, 5, 10, o] # no. of nodes 
L, E = 0.15, 210
#calling neural network function
weights = ANN_Classification(hidden_Layer_Size = layers, epochs=E, learning_Rate = L)

In [131]:
ans = weights.fit(X_train, Y_train)

In [133]:
from sklearn.metrics import accuracy_score
y_pred = weights.Predict(X_test)
accuracy_score(Y_test, y_pred)

0.8260869565217391

# **Using Keras**

In [101]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [116]:
# Build the model

model = Sequential()

model.add(Dense(10, input_shape=(4,), activation='sigmoid', name='h1'))
model.add(Dense(10, activation='sigmoid', name='h2'))
model.add(Dense(3, activation='softmax', name='output'))

# Adam optimizer with learning rate of 0.001
optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [118]:
model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3256 - accuracy: 0.9685
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3237 - accuracy: 0.9685
Epoch 3/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3218 - accuracy: 0.9685
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3202 - accuracy: 0.9685
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 0.3183 - accuracy: 0.9685
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 0.3167 - accuracy: 0.9685
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3150 - accuracy: 0.9685
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 0.3132 - accuracy: 0.9685
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 0.3116 - accuracy: 0.9685
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 0.3098 - accuracy: 0.9685
Epoch 11/100
4/4 [===========

In [134]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 221ms/step - loss: 0.2203 - accuracy: 0.9565


[0.2202633172273636, 0.95652174949646]